## En el siguiente Notebook se encuentra todo el codigo ejecutado para obtener el dataset final para el modelo ##

In [ ]:
import pandas as pd
import re 
from fuzzywuzzy import fuzz,process
import functools

**Se Importan los dataset para el inicio del filtrado y exploracion de los datos** 

In [ ]:
df = pd.read_csv("../dataset.csv", low_memory=False)
test = pd.read_csv("../ciudades_categorias.csv", low_memory=False)

**Se doprean las columnas innecesarias para este modelo**


Estas columnas no representan ningun valor para este segundo modelo el cual va a concentrar su entrenamientos en la columana de atributos.

In [ ]:
df.drop("name",axis="columns",inplace=True)
df.drop("city",axis="columns",inplace=True)
df.drop("review_count",axis="columns",inplace=True)

*Se categorizan variables para optimizar el dataset de entrada*

In [ ]:
df["categoria"] = pd.Categorical(df["categoria"]).codes
df["state"] = pd.Categorical(df["state"]).codes

**Ahora se comienza a dropear nulos y esatandarizar categorias y atributos**

In [ ]:
df.dropna(subset=["atributos"], inplace=True)
df.isnull().sum()

In [ ]:
df["atributos"].nunique()

In [ ]:
df["categoria"].nunique()

In [ ]:
catunique =[]
catunique = test["NDescripcion"].unique().tolist()

In [ ]:
len(catunique)

*La siguiente función ayuda a filtrar categorias repetidas o de mismo valor a partir de la lista generada previamente*

In [ ]:
def calcular_umbral_similitud(longitud_ciudad):
    umbral_base = 70
    umbral = umbral_base - (longitud_ciudad // 4)
    return max(umbral, umbral_base)

@functools.lru_cache(maxsize=None)
def encontrar_mejor_coincidencia(ciudad):
    mejor_coincidencia = process.extractOne(ciudad, catunique)
    resultado = mejor_coincidencia[0] if mejor_coincidencia[1] >= calcular_umbral_similitud(len(ciudad)) else ciudad
    return resultado

*Se convierten los valores para poder ser procesados por la función*

In [ ]:
df['categoria'] = df['categoria'].astype(str)
df["categoria"] = df["categoria"].str.lstrip()
df["categorias_filtradas"] = df["categoria"].apply(encontrar_mejor_coincidencia)

El siguiente filtro permite visualizar las primeras 169 categorias con mas coincidencias

In [ ]:
df["categorias_filtradas"].value_counts(ascending=False).head(169)

Se filtra la nueva columna filtrada por las primeras 169 categorias con mejor coincidencia para luego ser reemplazos los valores de la columna original "categoria". Posteriormente, se dropea la columna generada. (Esto se hizo para comparar los valores de cada columna)

In [ ]:
top_categorias = df["categorias_filtradas"].value_counts(ascending=False).head(169).index.tolist()
df = df[df["categorias_filtradas"].isin(top_categorias)]
df["categoria"] = df["categorias_filtradas"]
df.drop("categorias_filtradas",axis="columns",inplace=True)

Se controla la cantidad de categorias

In [ ]:
df["categoria"].nunique()

**A patir de aca, se empieza a limpiar los atributos que no hacen un aporte significativo a la muestra y la optimizacion y nomralizacion de los parametros de entrada en la columna "atributos"**

In [ ]:
#Aca se filtran los valores que contengan ":" dos veces
atributos = df['atributos']
atributos_con_u = [atributo for atributo in atributos if isinstance(atributo, str) and atributo.split(':')[1].strip().startswith('u')]
for atributo in atributos_con_u:
    print(atributo)

In [ ]:
#Aca se filtran los valores que contengan "u" despues de los ":"
atributos_filtrados = []

for atributo in atributos:
    if isinstance(atributo, str):
        partes = atributo.split(':')
        if len(partes) == 2: 
            clave = partes[0].strip()
            valor = partes[1].strip().lstrip('u')
            atributos_filtrados.append(f'{clave}: {valor}')


In [ ]:
#Se crea una funcion de limpieza para atributos que burlaron el filtro anterior 
def limpiar_atributo(atributo):
    if isinstance(atributo, str):
        partes = atributo.split(':')
        if len(partes) == 2:
            clave = partes[0].strip()
            valor = partes[1].strip().lstrip('u')
            return f'{clave}: {valor}'
    return atributo

In [ ]:
df['atributos'] = df['atributos'].apply(limpiar_atributo)

In [ ]:
#Se crea una funcion de limpieza para atributos que contienen un espacio vacio o caracteres desconocidos
def limpiar_atributo2(atributo):
    if isinstance(atributo, str):
        atributo_limpio = re.sub(r':\su\w+', '', atributo)
        return atributo_limpio.strip()
    return atributo

In [ ]:
df['atributos'] = df['atributos'].apply(limpiar_atributo2)

In [ ]:
#Se vuelve a correr el filtro como medida final debido a atributos que aparecian de nuevo
atributos = df['atributos']
atributos_con_u = [atributo for atributo in atributos if isinstance(atributo, str) and atributo.split(':')[1].strip().startswith('u')]
for atributo in atributos_con_u:
    print(atributo)

In [ ]:
#Esta operacion controla la existencia de valores que hayan podido saltar los filtros de limpieza
suma = sum(float(atributo.split(':')[1].strip()) for atributo in atributos_con_u if atributo.split(':')[1].strip().isdigit())
suma

In [ ]:
#print de control
print(df['atributos'].to_string(index=False))

In [ ]:
#Se agrega un ultimo filtro endswitch para eliminar definitivamente los "False"
df = df[~df['atributos'].str.endswith('False')]

In [ ]:
#Se agrega un ultimo filtro endswitch para eliminar definitivamente los "False"
consulta = df[df['atributos'].apply(lambda x: str(x).endswith("False"))]
consulta

In [ ]:
#Se agrega un ultimo filtro endswitch para eliminar definitivamente los espacios vacios 
df = df[~df['atributos'].str.contains(': None$|:$', regex=True)]

In [ ]:
#Head de control
df.head()

In [ ]:
#Se controla la cantidad final de categorias para coincidir con el modelo de raiting antes de su entrenamiento
df["categoria"].nunique()

Se exporta el dataset limpio para su entrenamiento (comentar/descomentar despues de usar)

In [ ]:
# df.to_csv("dataset2.csv", index=False) 